In [1]:
import chardet

# 바이너리 모드로 파일 열기
with open("C:/Users/Yeji/Desktop/열람실팟/학원교습소/서울시 학원 교습소정보.csv", 'rb') as f:
    result = chardet.detect(f.read())
    print(result)

{'encoding': 'CP949', 'confidence': 0.99, 'language': 'Korean'}


In [18]:
import pandas as pd

# CSV 파일 읽기 (인코딩 방식 지정)
df = pd.read_csv("C:/Users/Yeji/Desktop/열람실팟/학원교습소/서울시 학원 교습소정보.csv", encoding='cp949')

# 강남구 데이터 필터링
gangnam = df[df['행정구역명'] == '강남구']
# 새 CSV 파일로 저장
gangnam.to_csv("C:/Users/Yeji/Desktop/열람실팟/학원교습소/서울시 학원 교습소정보_강남구.csv", index=False, encoding='utf-8-sig')

# 송파구 데이터 필터링
songpa = df[df['행정구역명'] == '송파구']
# 새 CSV 파일로 저장
songpa.to_csv("C:/Users/Yeji/Desktop/열람실팟/학원교습소/서울시 학원 교습소정보_송파구.csv", index=False, encoding='utf-8-sig')

# 마포구 데이터 필터링
mapo = df[df['행정구역명'] == '마포구']
# 새 CSV 파일로 저장
mapo.to_csv("C:/Users/Yeji/Desktop/열람실팟/학원교습소/서울시 학원 교습소정보_마포구.csv", index=False, encoding='utf-8-sig')

In [19]:
gangnam.shape

(3613, 22)

In [26]:
missing = gangnam['도로명주소'].isna().sum()
print(missing)
missing_count = len(gangnam[gangnam['도로명주소'] == '.'])
print(missing_count)

7
17


In [42]:
import numpy as np
condition = gangnam['도로명주소'].isna()
position = np.where(condition)[0]
print(position+2)
condition2 = (gangnam['도로명주소'] == '.')
position2 = np.where(condition2)[0]
position2+2

[ 322  324  325  326  334  351 3599]


array([328, 330, 339, 340, 341, 342, 343, 345, 346, 348, 349, 350, 352,
       387, 389, 393, 405])

'도로명주소'가 결측치 or '.'인 행 데이터 제거

In [ ]:
filtered = gangnam[(gangnam['도로명주소'].notna()) & (gangnam['도로명주소'] != '.')]

# 새로운 CSV 파일로 저장
filtered.to_csv("C:/Users/Yeji/Desktop/열람실팟/학원교습소/서울시 학원 교습소정보_강남구.csv", index=False, encoding='utf-8-sig')

In [27]:
missing = songpa['도로명주소'].isna().sum()
print(missing)
missing_count = len(songpa[songpa['도로명주소'] == '.'])
print(missing_count)

0
0


In [28]:
missing = mapo['도로명주소'].isna().sum()
print(missing)
missing_count = len(mapo[mapo['도로명주소'] == '.'])
print(missing_count)

0
0


In [46]:
import chardet

# 바이너리 모드로 파일 열기
with open("C:/Users/Yeji/Desktop/열람실팟/학원교습소/서울시 학원 교습소정보_강남구.csv", 'rb') as f:
    result = chardet.detect(f.read())
    print(result)

{'encoding': 'UTF-8-SIG', 'confidence': 1.0, 'language': ''}


In [ ]:
gangnam = filtered.rename(columns={'도로명주소': '주소'})
gangnam.to_csv("C:/Users/Yeji/Desktop/열람실팟/학원교습소/서울시 학원 교습소정보_강남구.csv", encoding='utf-8-sig', index=False)


In [ ]:
import requests
import pandas as pd
import numpy as np

def doro_to_coords(df):
    data = df.copy()
    data['경도'] = 0
    data['위도'] = 0
    
    for i in range(len(data)):
        if pd.notna(data.loc[i, '주소']) and data.loc[i, '주소'] != '탈퇴':  # '도로명주소' → '주소' 변경
            try:
                address = data.loc[i, '주소']
                url = f'https://dapi.kakao.com/v2/local/search/address.json?query={address}'
                headers = {"Authorization": "KakaoAK " + "bbda3405a8e26906501ca18e70286239"}
                response = requests.get(url, headers=headers).json()
                
                if response.get('documents'):
                    temp = response['documents'][0].get('road_address')  # 도로명주소 정보 가져오기
                    if temp:
                        data.loc[i, ['경도', '위도']] = [temp['x'], temp['y']]
            
            except Exception as e:
                print(f"오류 발생: {e} (행 {i})")
    
    return data

# CSV 파일 읽기 (경로 수정)
df = pd.read_csv("C:/Users/Yeji/Desktop/열람실팟/학원교습소/서울시 학원 교습소정보_강남구.csv", encoding='utf-8-sig')
# 함수 적용
result_df = doro_to_coords(df)
# 결과 저장 (경로 수정)
result_df.to_csv("C:/Users/Yeji/Desktop/열람실팟/학원교습소/서울시 학원 교습소정보_강남구_최종.csv", encoding='utf-8-sig', index=False)


C:\Users\Yeji\AppData\Local\Temp\ipykernel_11544\2491252872.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '127.051899209649' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[i, ['경도', '위도']] = [temp['x'], temp['y']]
C:\Users\Yeji\AppData\Local\Temp\ipykernel_11544\2491252872.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '37.4990331412977' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[i, ['경도', '위도']] = [temp['x'], temp['y']]


In [ ]:
# CSV 파일 읽기 (경로 수정)
df = pd.read_csv("C:/Users/Yeji/Desktop/열람실팟/학원교습소/서울시 학원 교습소정보_강남구.csv", encoding='utf-8-sig')
# 함수 적용
result_df = doro_to_coords(df)
# 결과 저장 (경로 수정)
result_df.to_csv("C:/Users/Yeji/Desktop/열람실팟/학원교습소/서울시 학원 교습소정보_강남구.csv", encoding='utf-8-sig', index=False)

C:\Users\Yeji\AppData\Local\Temp\ipykernel_11544\2491252872.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '127.051899209649' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[i, ['경도', '위도']] = [temp['x'], temp['y']]
C:\Users\Yeji\AppData\Local\Temp\ipykernel_11544\2491252872.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '37.4990331412977' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[i, ['경도', '위도']] = [temp['x'], temp['y']]


오류 발생: HTTPSConnectionPool(host='dapi.kakao.com', port=443): Max retries exceeded with url: /v2/local/search/address.json?query=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%20%EA%B0%95%EB%82%A8%EA%B5%AC%20%EC%95%95%EA%B5%AC%EC%A0%95%EB%A1%9C29%EA%B8%B8%2068 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002BBC535B950>, 'Connection to dapi.kakao.com timed out. (connect timeout=None)')) (행 590)
오류 발생: HTTPSConnectionPool(host='dapi.kakao.com', port=443): Max retries exceeded with url: /v2/local/search/address.json?query=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%20%EA%B0%95%EB%82%A8%EA%B5%AC%20%EC%84%A0%EB%A6%89%EB%A1%9C%2030 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002BBC7851D30>, 'Connection to dapi.kakao.com timed out. (connect timeout=None)')) (행 2137)


In [ ]:
# CSV 파일 읽기
df = pd.read_csv("C:/Users/Yeji/Desktop/열람실팟/학원교습소/서울시 학원 교습소정보_강남구.csv", encoding='utf-8-sig')

# 전체 데이터 수 계산
total_records = len(df)

# '위도' 열의 값이 0인 데이터 수 계산
zero_latitude = (df['위도'] == 0).sum()

# 결과 출력
print(f"전체 레코드 수: {total_records}")
print(f"위도 값이 0인 데이터 수: {zero_latitude}")
print(f"위도 값이 0인 데이터 비율: {zero_latitude/total_records*100:.2f}%")

전체 레코드 수: 3589
위도 값이 0인 데이터 수: 17
위도 값이 0인 데이터 비율: 0.47%


4326 -> 5179

In [11]:
import pandas as pd
import numpy as np

# CSV 파일 읽기
df = pd.read_csv("C:/Users/Yeji/Desktop/열람실팟/학원교습소/서울시 학원 교습소정보_송파구.csv", encoding='utf-8-sig')

import geopandas as gpd
from shapely.geometry import Point

# 조건 마스크 생성 (원본 좌표가 모두 0인 경우)
zero_mask = (df['경도'] == 0) & (df['위도'] == 0)

# GeoDataFrame 생성
geometry = [Point(lon, lat) for lon, lat in zip(df['경도'], df['위도'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')

# 좌표계 변환 (0이 아닌 데이터만 변환 수행)
gdf_transformed = gdf.copy()
if not zero_mask.all():  # 0이 아닌 데이터가 있는 경우에만 변환
    gdf_non_zero = gdf[~zero_mask].to_crs('EPSG:5179')
    gdf_transformed.loc[~zero_mask, 'geometry'] = gdf_non_zero.geometry

# 좌표값 업데이트 (조건부 할당)
gdf_transformed['경도'] = np.where(zero_mask, 0, gdf_transformed.geometry.x)
gdf_transformed['위도'] = np.where(zero_mask, 0, gdf_transformed.geometry.y)

df1 = gdf_transformed[['주소', '위도', '경도']]

# CSV 저장
df1.to_csv("C:/Users/Yeji/Desktop/열람실팟/학원교습소/서울시 학원 교습소정보_송파구.csv", encoding='utf-8-sig', index=False)

위경도 값이 0인 데이터 필터링

In [10]:
import pandas as pd
df = pd.read_csv("C:/Users/Yeji/Desktop/열람실팟/학원교습소/서울시 학원 교습소정보_송파구.csv", encoding='utf-8-sig')

# 위도가 0인 행만 필터링
filtered_df = df[df['위도'] == 0]
filtered_df1 = filtered_df[['주소']]
filtered_df2 = filtered_df1.rename(columns={'주소': 'addr'})

# 필터링된 데이터를 새 CSV 파일로 저장
filtered_df2.to_csv("C:/Users/Yeji/Desktop/열람실팟/학원교습소/서울시 학원 교습소정보_송파구0.csv", encoding='utf-8-sig', index=False)

# 위도가 0이 아닌 행만 필터링
df_not_zero = df[df['위도'] != 0]
filtered_df3 = df_not_zero[['주소', '위도', '경도']]

# 필터링된 데이터를 새 CSV 파일로 저장
filtered_df3.to_csv("C:/Users/Yeji/Desktop/열람실팟/학원교습소/서울시 학원 교습소정보_송파구X.csv", encoding='utf-8-sig', index=False)


In [11]:
import pandas as pd
df1 = pd.read_csv("C:/Users/Yeji/Desktop/열람실팟/학원교습소/서울시 학원 교습소정보_송파구0.csv", encoding='utf-8-sig')
df2 = pd.read_csv("C:/Users/Yeji/Desktop/열람실팟/학원교습소/서울시 학원 교습소정보_송파구X.csv", encoding='utf-8-sig')
concatenated_df = pd.concat([df1, df2], ignore_index=True)
concatenated_df.to_csv("C:/Users/Yeji/Desktop/열람실팟/학원교습소/서울시 학원 교습소정보_송파구_최종.csv", encoding='utf-8-sig', index=False)

In [5]:
df.shape

(3589, 3)

In [3]:
filtered_df.shape

(17, 3)

In [4]:
df_not_zero.shape

(3572, 3)

In [56]:
songpa = songpa.rename(columns={'도로명주소': '주소'})
songpa.to_csv("C:/Users/Yeji/Desktop/열람실팟/학원교습소/서울시 학원 교습소정보_송파구.csv", encoding='utf-8-sig', index=False)

mapo = mapo.rename(columns={'도로명주소': '주소'})
mapo.to_csv("C:/Users/Yeji/Desktop/열람실팟/학원교습소/서울시 학원 교습소정보_마포구.csv", encoding='utf-8-sig', index=False)

In [57]:
# CSV 파일 읽기 (경로 수정)
df = pd.read_csv("C:/Users/Yeji/Desktop/열람실팟/학원교습소/서울시 학원 교습소정보_송파구.csv", encoding='utf-8-sig')
# 함수 적용
result_df = doro_to_coords(df)
# 결과 저장 (경로 수정)
result_df.to_csv("C:/Users/Yeji/Desktop/열람실팟/학원교습소/서울시 학원 교습소정보_송파구_최종.csv", encoding='utf-8-sig', index=False)


C:\Users\Yeji\AppData\Local\Temp\ipykernel_11544\2491252872.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '127.101917258808' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[i, ['경도', '위도']] = [temp['x'], temp['y']]
C:\Users\Yeji\AppData\Local\Temp\ipykernel_11544\2491252872.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '37.5178866228576' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[i, ['경도', '위도']] = [temp['x'], temp['y']]


In [58]:
# CSV 파일 읽기
df = pd.read_csv("C:/Users/Yeji/Desktop/열람실팟/학원교습소/서울시 학원 교습소정보_송파구_최종.csv", encoding='utf-8-sig')

# 전체 데이터 수 계산
total_records = len(df)

# '위도' 열의 값이 0인 데이터 수 계산
zero_latitude = (df['위도'] == 0).sum()

# 결과 출력
print(f"전체 레코드 수: {total_records}")
print(f"위도 값이 0인 데이터 수: {zero_latitude}")
print(f"위도 값이 0인 데이터 비율: {zero_latitude/total_records*100:.2f}%")

전체 레코드 수: 1972
위도 값이 0인 데이터 수: 2
위도 값이 0인 데이터 비율: 0.10%


In [59]:
# CSV 파일 읽기 (경로 수정)
df = pd.read_csv("C:/Users/Yeji/Desktop/열람실팟/학원교습소/서울시 학원 교습소정보_마포구.csv", encoding='utf-8-sig')
# 함수 적용
result_df = doro_to_coords(df)
# 결과 저장 (경로 수정)
result_df.to_csv("C:/Users/Yeji/Desktop/열람실팟/학원교습소/서울시 학원 교습소정보_마포구_최종.csv", encoding='utf-8-sig', index=False)


C:\Users\Yeji\AppData\Local\Temp\ipykernel_11544\2491252872.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '126.945190775648' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[i, ['경도', '위도']] = [temp['x'], temp['y']]
C:\Users\Yeji\AppData\Local\Temp\ipykernel_11544\2491252872.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '37.5564397859151' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[i, ['경도', '위도']] = [temp['x'], temp['y']]


In [60]:
# CSV 파일 읽기
df = pd.read_csv("C:/Users/Yeji/Desktop/열람실팟/학원교습소/서울시 학원 교습소정보_마포구_최종.csv", encoding='utf-8-sig')

# 전체 데이터 수 계산
total_records = len(df)

# '위도' 열의 값이 0인 데이터 수 계산
zero_latitude = (df['위도'] == 0).sum()

# 결과 출력
print(f"전체 레코드 수: {total_records}")
print(f"위도 값이 0인 데이터 수: {zero_latitude}")
print(f"위도 값이 0인 데이터 비율: {zero_latitude/total_records*100:.2f}%")

전체 레코드 수: 1164
위도 값이 0인 데이터 수: 4
위도 값이 0인 데이터 비율: 0.34%
